In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import omegaconf
from shell.utils.experiment_utils import *
from shell.fleet.utils.fleet_utils import *
from shell.utils.metric import *
import matplotlib.pyplot as plt
from shell.fleet.network import TopologyGenerator
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd
from shell.fleet.fleet import Agent, Fleet
from shell.fleet.data.data_utilize import *
from shell.fleet.data.recv import *

from sklearn.manifold import TSNE
from torchvision.utils import make_grid
from shell.fleet.data.data_utilize import *
import logging
from sklearn.metrics import f1_score
import os
from shell.fleet.data.recv_utils import *
from tqdm import tqdm
import argparse
from functools import partial
from torchvision.utils import make_grid
from shell.utils.oodloss import OODSeparationLoss
from pythresh.thresholds.dsn import DSN
from pythresh.thresholds.aucp import AUCP
from pythresh.thresholds.boot import BOOT
from pythresh.thresholds.zscore import ZSCORE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from collections import defaultdict
from shell.learners.fl_utils import *
import copy

from shell.fleet.network import *
logging.basicConfig(level=logging.INFO)

In [21]:
experiment_folder = "."
# experiment_folder = "rerun_opt_heuristic_experiment_results"
# experiment_name = "more_fl_fix_root_agent_results/fedprox_mu_0.1_comm_freq_5"
experiment_name = "best_fl_results/fedprox_mu_0.001_comm_freq_5"
dataset = "cifar100"
algo = "modular"
# algo = "monolithic"

use_contrastive = False
num_trains_per_class = 64 if dataset != "cifar100" else 256
seed = 0
parallel = False
task_id = 4

In [22]:
modify_cfg=None

In [23]:
save_dir = get_save_dir(experiment_folder, experiment_name,
                        dataset, algo, num_trains_per_class, use_contrastive, seed)

fleet = setup_fleet(save_dir, task_id=task_id, parallel=False, modify_cfg=modify_cfg)
fleet.change_save_dir('debug_more_fl_results')

{'train': {'component_update_freq': 100, 'num_epochs': 100, 'init_component_update_freq': 100, 'init_num_epochs': 100, 'save_freq': 10}, 'dataset': {'dataset_name': 'cifar100', 'num_tasks': 20, 'num_classes_per_task': 5, 'with_replacement': False, 'num_trains_per_class': 256, 'num_vals_per_class': -1, 'remap_labels': True}, 'net': {'name': 'cnn', 'depth': 4, 'channels': 50, 'conv_kernel': 3, 'maxpool_kernel': 2, 'padding': 1, 'dropout': 0.5, 'no_sparse_basis': True}, 'sharing_strategy': {'name': 'fedprox', 'num_coms_per_round': 1, 'comm_freq': 5, 'mu': 0.001, 'when_reoptimize_structure': 'never', 'sync_base': True, 'pre_or_post_comm': 'pre'}, 'seed': 0, 'algo': 'modular', 'job_name': 'cifar100_modular_numtrain_256', 'num_agents': 8, 'root_save_dir': 'best_fl_results/fedprox_mu_0.001_comm_freq_5', 'parallel': True, 'num_init_tasks': 4, 'overwrite': False, 'topology': 'fully_connected', 'edge_drop_prob': 0.0, 'agent': {'save_dir': '${root_save_dir}/${job_name}/${dataset.dataset_name}/${a

INFO:root:Class sequence: [97 66 33 19 34  3 12  2 13 92 80  5 27 60 36 11  9 25 31 87 58 44 94 95
 64 61 62 86 79 98 93 51 22 59 26 37 83 73 78  1 30 55 85 47 41 20 63 43
 90 88 21 40 15 75  4  0 52 23 35 53 84 69 16 10 71 81  6 91 56 76 38 50
 57 74 49 18 24 39 48 99  8 65 89 67 54 32 42 14 68 45 96 77 46 28 72 17
 29 82  7 70]
INFO:root:task 0 :(1280, 3, 32, 32) (469, 3, 32, 32) (500, 3, 32, 32)
INFO:root:task 1 :(1280, 3, 32, 32) (527, 3, 32, 32) (500, 3, 32, 32)
INFO:root:task 2 :(1280, 3, 32, 32) (499, 3, 32, 32) (500, 3, 32, 32)
INFO:root:task 3 :(1280, 3, 32, 32) (532, 3, 32, 32) (500, 3, 32, 32)
INFO:root:task 4 :(1280, 3, 32, 32) (495, 3, 32, 32) (500, 3, 32, 32)
INFO:root:task 5 :(1280, 3, 32, 32) (496, 3, 32, 32) (500, 3, 32, 32)
INFO:root:task 6 :(1280, 3, 32, 32) (503, 3, 32, 32) (500, 3, 32, 32)
INFO:root:task 7 :(1280, 3, 32, 32) (510, 3, 32, 32) (500, 3, 32, 32)
INFO:root:task 8 :(1280, 3, 32, 32) (506, 3, 32, 32) (500, 3, 32, 32)
INFO:root:task 9 :(1280, 3, 32, 32) (4

labels: [97 66 33 19 34  3 12  2 13 92 80  5 27 60 36 11  9 25 31 87 58 44 94 95
 64 61 62 86 79 98 93 51 22 59 26 37 83 73 78  1 30 55 85 47 41 20 63 43
 90 88 21 40 15 75  4  0 52 23 35 53 84 69 16 10 71 81  6 91 56 76 38 50
 57 74 49 18 24 39 48 99  8 65 89 67 54 32 42 14 68 45 96 77 46 28 72 17
 29 82  7 70]


INFO:root:task 11 :(1280, 3, 32, 32) (488, 3, 32, 32) (500, 3, 32, 32)
INFO:root:task 12 :(1280, 3, 32, 32) (490, 3, 32, 32) (500, 3, 32, 32)
INFO:root:task 13 :(1280, 3, 32, 32) (506, 3, 32, 32) (500, 3, 32, 32)
INFO:root:task 14 :(1280, 3, 32, 32) (504, 3, 32, 32) (500, 3, 32, 32)
INFO:root:task 15 :(1280, 3, 32, 32) (475, 3, 32, 32) (500, 3, 32, 32)
INFO:root:task 16 :(1280, 3, 32, 32) (471, 3, 32, 32) (500, 3, 32, 32)
INFO:root:task 17 :(1280, 3, 32, 32) (510, 3, 32, 32) (500, 3, 32, 32)
INFO:root:task 18 :(1280, 3, 32, 32) (532, 3, 32, 32) (500, 3, 32, 32)
INFO:root:task 19 :(1280, 3, 32, 32) (494, 3, 32, 32) (500, 3, 32, 32)
INFO:root:Class sequence: [13 55 29 67 64 47 58 19 30 38 15 16 34 35 73 96 25 89 63  5 44 21 28 60
 80 86 36 95 83 40  2 24 37 61 62 57 53 79 27 82 65  4 22 45 78 33 32  9
 90 56 46 48  7 59 41 93 98 69 51 74 68 72 71 39 54 18 91 76 70 11 10 99
 77 52 87 31  6  1 84 43 85 42 12 94 26 88 17 49 75 92  8 97 66 50  0 23
 20 81 14  3]
INFO:root:task 0 :(1280, 3, 3

labels: [13 55 29 67 64 47 58 19 30 38 15 16 34 35 73 96 25 89 63  5 44 21 28 60
 80 86 36 95 83 40  2 24 37 61 62 57 53 79 27 82 65  4 22 45 78 33 32  9
 90 56 46 48  7 59 41 93 98 69 51 74 68 72 71 39 54 18 91 76 70 11 10 99
 77 52 87 31  6  1 84 43 85 42 12 94 26 88 17 49 75 92  8 97 66 50  0 23
 20 81 14  3]


...

i_size 32
num_classes 5
net_cfg {'name': 'cnn', 'depth': 4, 'channels': 50, 'conv_kernel': 3, 'maxpool_kernel': 2, 'padding': 1, 'dropout': 0.5, 'no_sparse_basis': True, 'i_size': 32, 'num_classes': 5, 'num_tasks': 20, 'num_init_tasks': 4, 'use_contrastive': False}
<class 'shell.learners.er_dynamic.CompositionalDynamicER'>


INFO:root:Class sequence: [ 7  8 19 18 48 59 36 34 75 30 93 86 53  6  9 77 95 89 39 55 71 32 61 12
 50 99 74 96 24 69 49 57 47 78  1 62 73  0 37 40 70 65 27  5 83 98 91 51
 15 72 64 60 20 23 82 45 44 63 68 17 25 11 76 16  3 58 67  2 90 21 35 94
 14 28 56 22 85 81 26 79 66 54 92 52 43 97 87 10 80  4 33 46 29 88 84 41
 31 38 13 42]


labels: [ 7  8 19 18 48 59 36 34 75 30 93 86 53  6  9 77 95 89 39 55 71 32 61 12
 50 99 74 96 24 69 49 57 47 78  1 62 73  0 37 40 70 65 27  5 83 98 91 51
 15 72 64 60 20 23 82 45 44 63 68 17 25 11 76 16  3 58 67  2 90 21 35 94
 14 28 56 22 85 81 26 79 66 54 92 52 43 97 87 10 80  4 33 46 29 88 84 41
 31 38 13 42]


INFO:root:task 0 :(1280, 3, 32, 32) (473, 3, 32, 32) (500, 3, 32, 32)
INFO:root:task 1 :(1280, 3, 32, 32) (507, 3, 32, 32) (500, 3, 32, 32)
INFO:root:task 2 :(1280, 3, 32, 32) (533, 3, 32, 32) (500, 3, 32, 32)
INFO:root:task 3 :(1280, 3, 32, 32) (493, 3, 32, 32) (500, 3, 32, 32)
INFO:root:task 4 :(1280, 3, 32, 32) (506, 3, 32, 32) (500, 3, 32, 32)
INFO:root:task 5 :(1280, 3, 32, 32) (492, 3, 32, 32) (500, 3, 32, 32)
INFO:root:task 6 :(1280, 3, 32, 32) (511, 3, 32, 32) (500, 3, 32, 32)
INFO:root:task 7 :(1280, 3, 32, 32) (512, 3, 32, 32) (500, 3, 32, 32)
INFO:root:task 8 :(1280, 3, 32, 32) (481, 3, 32, 32) (500, 3, 32, 32)
INFO:root:task 9 :(1280, 3, 32, 32) (502, 3, 32, 32) (500, 3, 32, 32)
INFO:root:task 10 :(1280, 3, 32, 32) (501, 3, 32, 32) (500, 3, 32, 32)
INFO:root:task 11 :(1280, 3, 32, 32) (505, 3, 32, 32) (500, 3, 32, 32)
INFO:root:task 12 :(1280, 3, 32, 32) (470, 3, 32, 32) (500, 3, 32, 32)
INFO:root:task 13 :(1280, 3, 32, 32) (537, 3, 32, 32) (500, 3, 32, 32)
INFO:root:task 1

Setting seed 69420000
Setting seed 0
Setting seed 1000
Setting seed 2000
Setting seed 3000
Setting seed 4000
Setting seed 5000
Setting seed 6000
Setting seed 7000


In [24]:
agent = fleet.agents[0]
agent.eval_test(task_id, include_avg=True)

{0: 0.676, 1: 0.856, 2: 0.766, 3: 0.694, 4: 0.812, 'avg': 0.7608}

In [25]:
fleet.communicate(task_id, end_epoch=None, comm_freq=None, num_epochs=None)

INFO:root:AGGREGATING MODELS...no_components 5
INFO:root:AGGREGATING MODELS...no_components 5
INFO:root:AGGREGATING MODELS...no_components 5
INFO:root:AGGREGATING MODELS...no_components 5
INFO:root:AGGREGATING MODELS...no_components 5


INFO:root:AGGREGATING MODELS...no_components 5
INFO:root:AGGREGATING MODELS...no_components 5
INFO:root:AGGREGATING MODELS...no_components 5


In [26]:
agent.eval_test(task_id, include_avg=True)

{0: 0.718, 1: 0.852, 2: 0.806, 3: 0.712, 4: 0.798, 'avg': 0.7772}

In [27]:
(0.7772 * 4 + 0.62)/5

0.74576